# **1. Tải thư viện cần thiết**

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 

In [ ]:
import os
import torch
import bitsandbytes as bnb
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

# **2. Tải mô hình quantization hóa**

In [ ]:
# Tải mô hình
model_name = "vilm/vinallama-2.7b"

# Cấu hình 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load mô hình và tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# **3. Thử nghiệm mô hình gốc**

In [ ]:
# def generate_answer(question, max_new_tokens=125):
#     prompt = f"{question}"
    
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=max_new_tokens,
#             temperature=0.7,
#             top_p=0.9,
#             repetition_penalty=1.1,
#             do_sample=True,
#             eos_token_id=tokenizer.eos_token_id
#         )
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return answer


# question = "Bệnh cúm là gì?"
# answer = generate_answer(question)
# print(answer)

In [ ]:
#for name, module in model.named_modules():
#     if isinstance(module, torch.nn.Linear):
#         print(name, module)

# **4. Cấu hình LoRA**

In [ ]:
# Bật checkpointing + LoRA
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

**Kiểm tra thông tin mô hình**

In [ ]:
# print("\n========== Thông tin tổng quát về mô hình ==========")

# # Tổng số tham số
# total_params = sum(p.numel() for p in model.parameters())
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f"Tổng số tham số của mô hình gốc: ~ 2.7B ")
# print(f"Tổng số tham số của mô hình khi đã lượng tử hóa về 4-bit: {total_params:,}")
# print(f"Số tham số trainable khi áp dụng Fine-tunning với LoRA: {trainable_params:,}")

# # Thông tin về cấu hình mô hình
# config = model.config

# print(f"\nMô hình loại: {config.model_type}")
# print(f"Số layers (num_hidden_layers): {config.num_hidden_layers}")
# print(f"Số attention heads: {config.num_attention_heads}")
# print(f"Hidden size: {config.hidden_size}")
# print(f"Intermediate size: {config.intermediate_size}")
# print(f"Head dim: {config.head_dim}")
# print(f"Torch dtype: {config.torch_dtype}")
# print(f"Số chiều context tối đa (max_position_embeddings): {config.max_position_embeddings}")

# # Thông tin về vocab
# print(f"\nKích thước từ vựng (vocab_size): {config.vocab_size}")

# # Kiểm tra embedding dimension
# embed_weight = model.get_input_embeddings().weight
# print(f"Chiều vector embedding: {embed_weight.shape[1]}")

# # Thông tin về tokenizer
# print("\n========== Thông tin tokenizer ==========")
# print(f"Padding token id: {tokenizer.pad_token_id}")
# print(f"BOS token id: {tokenizer.bos_token_id}")
# print(f"EOS token id: {tokenizer.eos_token_id}")
# print(f"Special tokens: {tokenizer.special_tokens_map}")

# # In danh sách token đặc biệt
# print("\nDanh sách các token đặc biệt:")
# for name, token in tokenizer.special_tokens_map.items():
#     print(f"- {name}: {token}")

# **5. Fine-tuning LLM**

**5.1. Chuẩn bị dữ liệu**

In [ ]:
from datasets import load_dataset

# Load từng file CSV thành dataset độc lập  
train_data = load_dataset("csv", data_files={"train": "/kaggle/input/medical-qa-data-v2/disease_QA_v2_train.csv"})["train"]
val_data = load_dataset("csv", data_files={"validation": "/kaggle/input/medical-qa-data-v2/disease_QA_v2_val.csv"})["validation"]
test_data = load_dataset("csv", data_files={"test": "/kaggle/input/medical-qa-data-v2/disease_QA_v2_test.csv"})["test"]

# Print số lượng mẫu trong từng bộ dữ liệu
print("Số lượng dữ liệu train: ", len(train_data))
print("Số lượng dữ liệu val: ", len(val_data))
print("Số lượng dữ liệu test: ", len(test_data))


**Thống kê dữ liệu**

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Đọc file CSV
# df = pd.read_csv("/kaggle/input/disease-qa-data-v2/disease_QA_v2_train.csv")  

# # Hàm đếm số token
# def count_tokens(text):
#     return len(tokenizer.tokenize(str(text)))

# # Thêm cột mới cho số lượng token
# df["title_tokens"] = df["Title"].apply(count_tokens)
# df["question_tokens"] = df["Question"].apply(count_tokens)
# df["answer_tokens"] = df["Answer"].apply(count_tokens)

# # Tổng số token của mỗi dòng
# df["total_tokens"] = df["title_tokens"] + df["question_tokens"] + df["answer_tokens"]

# # ✅ Thống kê tổng quan
# print("\n📊 Thống kê:")
# print(df[["title_tokens", "question_tokens", "answer_tokens", "total_tokens"]].describe())

# # ✅ Biểu đồ phân phối tổng token
# plt.hist(df["total_tokens"], bins=50, color="skyblue")
# plt.title("Phân phối tổng số token mỗi dòng")
# plt.xlabel("Số lượng token")
# plt.ylabel("Số dòng")
# plt.grid(True)
# plt.show()

**Chuẩn hóa prompt huấn luyện**

In [ ]:
def format_prompt(example):
    return (
        "<s>### Hướng dẫn:\n"
        "Bạn là một trợ lý y tế AI đáng tin cậy. Hãy trả lời các câu hỏi về y tế, bệnh học, chăm sóc sức khỏe bằng tiếng Việt một cách ngắn gọn, dễ hiểu, và chính xác. "
        "Nếu câu hỏi nằm ngoài chuyên môn hoặc chưa có đủ dữ liệu khoa học, hãy trả lời rằng 'Hiện tại tôi chưa có đủ thông tin để trả lời câu hỏi này.'\n"
        "### Câu hỏi:\n"
        f"{example['Question']}\n"
        "### Câu trả lời:\n"
        f"{example['Answer']}</s>"
    )

train_data = train_data.map(lambda x: {"text": format_prompt(x)}, num_proc=4, desc="Formatting")
val_data = val_data.map(lambda x: {"text": format_prompt(x)}, num_proc=4, desc="Formatting")
test_data = test_data.map(lambda x: {"text": format_prompt(x)}, num_proc=4, desc="Formatting")

def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=280,
    )

**Chuẩn bị dữ liệu huấn luyện**

In [ ]:
train_tokenized = train_data.map(
    preprocess_function,
    remove_columns=train_data.column_names,
    num_proc=4,
    batched=True,
    desc="Tokenizing"
)
val_tokenized = val_data.map(
    preprocess_function,
    remove_columns=val_data.column_names,
    num_proc=4,
    batched=True,
    desc="Tokenizing"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
# # In thử sau khi tokenize
# sample_tokenized = train_tokenized[0]
# print(sample_tokenized)

# # In thử lại text đã decode từ input_ids
# decoded = tokenizer.decode(sample_tokenized["input_ids"])
# print(decoded)

# # Kiểm tra độ dài token
# print("Số token của mẫu:", len(sample_tokenized["input_ids"]))

# **6. Cấu hình huấn luyện**

In [ ]:
import csv
from transformers import TrainerCallback

class CSVLoggerCallback(TrainerCallback):
    def __init__(self, filename="training_log.csv"):
        self.filename = filename
        # Tạo header nếu file chưa tồn tại
        with open(self.filename, mode="w", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=["step", "epoch", "train_loss", "eval_loss", "learning_rate"])
            writer.writeheader()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            with open(self.filename, mode="a", newline="") as file:
                writer = csv.DictWriter(file, fieldnames=["step", "epoch", "train_loss", "eval_loss", "learning_rate"])
                row = {
                    "step": state.global_step,
                    "epoch": state.epoch,
                    "train_loss": logs.get("loss"),
                    "eval_loss": logs.get("eval_loss"),
                    "learning_rate": logs.get("learning_rate"),
                }
                writer.writerow(row)


In [ ]:
training_args = TrainingArguments(
    output_dir="./Medical_QA_villama2.7B_checkpoint",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=23,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    fp16=True,
    dataloader_num_workers=4,
    logging_steps=100,
    save_strategy="epoch",
    eval_strategy="epoch",  
    save_total_limit=1,
    optim="paged_adamw_8bit",
    report_to=[],
)

trainer = Trainer(
    model=model,  
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    callbacks=[CSVLoggerCallback("training_loss.csv")]
)

model.config.use_cache = False


**Huấn luyện mô hình**

In [ ]:
# print("Bắt đầu huấn luyện mô hình!")
# trainer.train()


**Huấn luyện tiếp từ checkpoint**

In [ ]:
checkpoint_path="/kaggle/input/medical-qa-model-vinallama2-7b-epoch-21/Medical_QA_villama2.7B_checkpoint/checkpoint-6846"
print("Bắt đầu huấn luyện mô hình từ check point")
trainer.train(resume_from_checkpoint=checkpoint_path)

In [ ]:
import pandas as pd

# Sau khi train xong
log_history = trainer.state.log_history
df = pd.DataFrame(log_history)
df.to_csv("training_log.csv", index=False)


**Lưu lại mô hình**

In [ ]:
trainer.save_model("Medical_QA_model_villma2.7B_epoch_23")
tokenizer.save_pretrained("Medical_QA_model_villma2.7B_epoch_23")

# **7. Dự doán**

In [ ]:
def build_prompt(question):
    return (
        "<s>### Hướng dẫn:\n"
        "Bạn là một trợ lý y tế AI đáng tin cậy. Hãy trả lời các câu hỏi về y tế, bệnh học, chăm sóc sức khỏe bằng tiếng Việt một cách ngắn gọn, dễ hiểu, và chính xác. "
        "Nếu câu hỏi nằm ngoài chuyên môn hoặc chưa có đủ dữ liệu khoa học, hãy trả lời rằng 'Hiện tại tôi chưa có đủ thông tin để trả lời câu hỏi này.'\n"
        "### Câu hỏi:\n"
        f"{question}\n"
        "### Câu trả lời:\n"
    )

def generate_answer(question, max_new_tokens=100):
    prompt = build_prompt(question)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
question ="Bệnh cúm là gì?"
answer = generate_answer(question)
print(answer)
# Cúm (Flu) là một dạng bệnh nhiễm vi rút cấp tính. 
# Bệnh phát triển khi vi rút cúm lây nhiễm và tấn công vào hệ hô hấp đường mũi, 
# cổ họng, các ống phế quản và có thể bao gồm cả phổi.

In [ ]:
question ="Viêm túi mật không do sỏi là gì?"
answer = generate_answer(question)
print(answer)
# Viêm túi mật không do sỏi là dạng viêm túi mật do rối loạn chức năng làm rỗng túi mật, 
# hoàn toàn không liên quan đến sỏi.

In [ ]:
question ="Chế độ ăn uống cho người bị Phì đại cuốn mũi?"
answer = generate_answer(question)
print(answer)
# Không có chế độ ăn uống riêng nào được khuyến nghị cho người mắc phì đại cuốn mũi. 
# Tuy nhiên, người bệnh nên tránh ăn các thức ăn gây dị ứng để phòng ngừa nguy
# cơ dị ứng mũi xoang làm nghiêm trọng thêm tình trạng bệnh.

In [ ]:
question ="Địa chỉ điều trị xoắn tinh hoàn uy tín?"
answer = generate_answer(question)
print(answer)
# Trung tâm Tiết niệu Thận học, Khoa Nam học – Tiết niệu của 
# Hệ thống BVĐK Tâm Anh quy tụ đội ngũ các chuyên gia đầu ngành, 
# Nội khoa và Ngoại khoa, giỏi chuyên môn, tận tâm.

# **8. Đánh giá mô hình**

In [2]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=071a03ebcb71774959b2a8c1d940babf04b8222d371a5b0ac5af839bcd4bb0ee
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [3]:
from peft import (
    PeftConfig,
    PeftModel,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer as AutoTokenizerLlama,
    BitsAndBytesConfig,
)
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
import torch
from evaluate import load as load_metric
import re

# 1. Load Model và Tokenizer
PEFT_MODEL = "/kaggle/input/medical-qa-model-vinallama2-7b-epoch-23/Medical_QA_model_villma2.7B_epoch_23"

peft_config = PeftConfig.from_pretrained(PEFT_MODEL)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model_llama = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer_llama = AutoTokenizerLlama.from_pretrained(PEFT_MODEL, trust_remote_code=True)
tokenizer_llama.pad_token = tokenizer_llama.eos_token
tokenizer_llama.padding_side = "right"

model_llama = PeftModel.from_pretrained(base_model_llama, PEFT_MODEL)
device_llama = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"======> Load Model Medical VinaLLaMA-2.7b to device: {device_llama}\n")
model_llama.eval()

2025-06-25 06:33:31.502426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750833211.712462      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750833211.770317      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

======> Load Model Medical VinaLLaMA-2.7b to device: cuda



In [14]:
# 2. Load Dữ liệu test
test_df = pd.read_csv("/kaggle/input/medical-qa-data-v2/disease_QA_v2_test.csv")
test_dataset = Dataset.from_pandas(test_df)

# 3. Hàm build prompt
def build_prompt_llama(question):
    return (
        "<s>### Hướng dẫn:\n"
        "Bạn là một trợ lý y tế AI đáng tin cậy. Hãy trả lời các câu hỏi về y tế, bệnh học, chăm sóc sức khỏe bằng tiếng Việt một cách ngắn gọn, dễ hiểu, và chính xác. "
        "Nếu câu hỏi nằm ngoài chuyên môn hoặc chưa có đủ dữ liệu khoa học, hãy trả lời rằng 'Hiện tại tôi chưa có đủ thông tin để trả lời câu hỏi này.'\n"
        "### Câu hỏi:\n"
        f"{question}\n"
        "### Câu trả lời:\n"
    )

def extract_answer_llama(full_text):
    match = re.search(r'### Câu trả lời:\n(.+?)(###|$)', full_text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return full_text.strip()

def ensure_complete_sentence_llama(answer):
    sentences = re.split(r'(?<=[.!?])\s+', answer.strip())
    if len(sentences) > 1:
        return ' '.join(sentences[:-1])  # loại bỏ câu cuối nếu nó chưa hoàn chỉnh
    return answer

# 4. Chuẩn bị list question & answer
questions = [row["Question"] for row in test_dataset]
references = [row["Answer"] for row in test_dataset]

# 5. Batch inference
BATCH_SIZE = 64  # tăng nếu đủ VRAM
predictions = []

for i in tqdm(range(0, len(questions), BATCH_SIZE), desc="Generating answers"):
    batch_questions = questions[i:i+BATCH_SIZE]
    prompts = [build_prompt_llama(q) for q in batch_questions]
    inputs = tokenizer_llama(
    prompts,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=280  
    ).to(device_llama)
    with torch.no_grad():
        outputs = model_llama.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,   # Không sampling, nhanh và ổn định hơn
            eos_token_id=tokenizer_llama.eos_token_id
        )
    batch_answers = tokenizer_llama.batch_decode(outputs, skip_special_tokens=True)
    for full_answer in batch_answers:
        extracted_answer = extract_answer_llama(full_answer)
        final_answer = ensure_complete_sentence_llama(extracted_answer)
        predictions.append(final_answer)

Generating answers: 100%|██████████| 17/17 [07:57<00:00, 28.08s/it]


In [18]:
# 6. Đánh giá BLEU và ROUGE
bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")

bleu_result = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
rouge_result = rouge_metric.compute(predictions=predictions, references=references, use_stemmer=True)

print(f"BLEU score: {bleu_result['score']:.2f}")
print("ROUGE scores:")
for key in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']:
    print(f"{key}: {rouge_result[key]:.4f}")


BLEU score: 33.25
ROUGE scores:
rouge1: 0.6865
rouge2: 0.3201
rougeL: 0.4971
rougeLsum: 0.4972



# **9. Chạy mô hình (inference)**

In [ ]:
# PEFT_MODEL = "/kaggle/input/medical-qa-model-vinallama2-7b-epoch-6/Medical_QA_model_vinallma2.7B_epoch_6"

# config = PeftConfig.from_pretrained(PEFT_MODEL)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16  
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True
# )

# tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL, trust_remote_code=True)
# # tokenizer.pad_token = tokenizer.eos_token
# # tokenizer.padding_side = "right"

# model = PeftModel.from_pretrained(base_model, PEFT_MODEL)

In [ ]:
# gen_config = model.generation_config
# gen_config.temperature = 1
# gen_config.top_p = 0.85
# gen_config.do_sample = True
# gen_config.repetition_penalty = 1.1
# gen_config.max_new_tokens = 50
# gen_config.pad_token_id = tokenizer.eos_token_id
# gen_config.eos_token_id = tokenizer.eos_token_id

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print("Bạn đang dùng: ",device)

# def generate_answer(prompt):
#     full_prompt = (
#         "<|im_start|>system\n"
#         "Bạn là một trợ lý y tế thông minh, trả lời các câu hỏi y tế, bệnh học và chăm sóc sức khỏe bằng tiếng Việt, ngắn gọn, dễ hiểu, và chính xác. Nếu không biết câu trả lời, hãy thành thật trả lời là không biết.\n"
#         "<|im_end|>\n"
#         "<|im_start|>user\n"
#         f"{prompt}<|im_end|>\n"
#         "<|im_start|>assistant"
#     )
#     inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
#     outputs = model.generate(
#         **inputs,
#         generation_config=gen_config,
#     )
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     # Lấy phần sau cùng (phần mô hình sinh tiếp sau prompt)
#     answer = response.split("<|im_start|> assistant\n")[-1].strip()
#     return response

In [ ]:
# question ="Tôi chụp MRI và phát hiện có khối u trong tủy sống. Tôi có thể đang bị bệnh gì?   "
# answer = generate_answer(question)
# print(answer)
# # Cúm (Flu) là một dạng bệnh nhiễm vi rút cấp tính. Bệnh phát triển khi vi rút cúm lây nhiễm và 
# # tấn công vào hệ hô hấp đường mũi, cổ họng, các ống phế quản và có thể bao gồm cả phổi.